In [ ]:
#= Costanti (sola lettura)
-- Numbers and boolean --
number_of_steps = 1000; # Number of steps to execute in simulation, Intero positivo [1,+∞)
dim = 3; # Dimensions of simulation, dimensioni dello spazio 2D o 3D, Intero positivo (tipicamente [2,3])
box_size = 10.0; # Size of one side of box, lunghezza di un lato del campo di simulazione Float positivo [1.0, +∞)
finite_box = true; # Whether box if finite or just where particles are initially placed, vero se il campo è finito<APPROFONDIRE
periodic = true; # Whether simulation is periodic < true se la simulazione è periodica (pac-man)
part_num = 100; # Number of particles in simulation, numero di particelle da simulare 
dt = 0.01; # Time step, controlla la precisione della simulazione
num_part_types = 2; # Number of types of particles, numero di tipologie di particelle Intero positivo [1, +∞]
save_interval = 10; # Ogni quanti passi viene salvata la posizione, (per la visualizzazione), Intero positivo [1, number_of_steps)

-- Arrays --
interaction_params = gen_interaction(num_part_types); # Interations parameters for types of particles, parametri di interazione, array?
mass_parts = rand(Float64, num_part_types); # Masses of types of particles, massa per ogni tipo di particella, array
part_types = rand(1:num_part_types, part_num) # Randomly assign type of each particle
=#

In [ ]:
#= Strutture dati (lettura, scrittura)

# Matrici bi-dimensionali per tracciare lo stato del sistema ad ogni step
pos = box_size*rand(Float64, part_num, dim) # Initialized to be randomly placed within a box
vel = zeros(Float64, part_num, dim) # Initialized to zero
acc = zeros(Float64, part_num, dim) # Initialized to zero
force = find_force(....) # 

# Matrici tridimensionali per salvare lo stato del sistema ad ogni intervallo
saved_positions = zeros(num_pos, part_num, dim) # Save positions with specified frequency, unico utile per la visualizzazione
saved_velocities = zeros(num_pos, part_num, dim) # Save velocities with specified frequency
saved_accelerations = zeros(num_pos, part_num, dim) # Save accelerations with specified frequency
saved_forces = zeros(num_pos, part_num, dim) # Save forces with specified frequency
=#

In [1]:
# Generate strengths of interactions between particle types
# Very simple but user could replace with anything they wanted
# Modifiche: Cambio tipo da Float64 a Float32, inserito break perché matrice simmetrica
function gen_interaction(num_part_types)
    interaction_params = Array{Float32}(undef, num_part_types, num_part_types)
    for i=1:num_part_types
        for j = 1:num_part_types
            if (i==j) # Self-interaction is randomly repulsive
                interaction_params[i,j] = -1*rand(Float32)
            elseif (i<j) # Others randomly attractive
                val = rand(Float32)
                interaction_params[i,j] = val
                interaction_params[j,i] = val
            else
                break
            end
        end        
    end    
    return interaction_params
end

gen_interaction (generic function with 1 method)

In [3]:
# Test #
gen_interaction(10);
@time gen_interaction(10)

  0.000003 seconds (5 allocations: 656 bytes)


10×10 Array{Float32,2}:
 -0.876137   0.810953     0.330925     …  0.0502217    0.525466   
  0.810953   0.0          0.0             0.0          0.0        
  0.330925   5.04289e-29  5.08828e-29     5.04327e-29  5.04332e-29
  0.627507   0.0          0.0             0.0          0.0        
  0.483399   5.0429e-29   5.08836e-29     5.04328e-29  5.04333e-29
  0.669534   0.0          0.0          …  0.0          0.0        
  0.41113    5.04291e-29  5.07717e-29     5.04329e-29  5.04334e-29
  0.333238   0.0          0.0             0.0          0.0        
  0.0502217  5.04292e-29  5.07718e-29     5.0433e-29   5.04335e-29
  0.525466   0.0          0.0             0.0          0.0        

In [11]:
# Find force on each particle
# 1/r^2 interactions: Is very simple but user can replace with anything they want
function find_force!(force, part_num, dim, pos, vel, acc, part_types, interaction_params, mass_parts, periodic, box_size)    
    for i = 1:part_num # For every particle
        mass = mass_parts[Int(part_types[i])]
        for k = 1:part_num # Contribution from every other particle
            if (i != k) # No self-interaction
                for j = 1:dim # For each dimension
                    int_strength = interaction_params[part_types[i],part_types[k]] # Strength of interaction between particles
                    if (pos[i,j] > pos[k,j])
                       int_strength = -1*int_strength # Reverses direction of force if positions flopped
                    end

                    # Find distance between particles
                    dist = 0.0
                    if (periodic) # If periodic, check whether a periodic distance is shorter
                        dist = abs(pos[i,j] - pos[k,j])
                        if (pos[i,j] < pos[k,j])
                            new_dist = abs(box_size + pos[i,j] - pos[k,j])
                            if new_dist > dist
                                dist = new_dist
                                int_strength = -1*int_strength # Reverses direction of force
                            end
                        else
                            new_dist = abs(box_size + pos[k,j] - pos[i,j])
                            if new_dist > dist
                                dist = new_dist
                                int_strength = -1*int_strength # Reverses direction of force
                            end
                        end
                        else # Otherwise, just regular distance
                        dist = abs(pos[i,j] - pos[k,j])
                    end
                    
                    force[i,j] += int_strength / dist^2 # 1/r^2 interaction
                end
            end
        end
    end
    return nothing # Modifica C++ style
end

find_force! (generic function with 1 method)

In [16]:
# Update position, velocity, and acceleration using Velocity Verlet Algorithm
# Can deal with infinite and finite systems
# For finite system, can be periodic or can reflect off walls
function step_update!(part_num, dim, pos, vel, acc, force, part_types, mass_parts, dt, box_size, finite_box, periodic)

    # Loop is completely parallelizale
    for i = 1:part_num # For every particle
        mass = mass_parts[part_types[i]]
        for j = 1:dim # For each dimension
            pos[i,j] = pos[i,j] + vel[i,j]*dt + 0.5*acc[i,j]*dt^2 # x(t+Δt) = x(t) + v(t)Δt + 1/2*a(t)(Δt)^2
            vel[i,j] = vel[i,j] + 0.5*(acc[i,j] + force[i,j]/mass)*dt # v(t+Δt) = v(t) + 1/2*(a(t)+a(t+Δt))Δt
            acc[i,j] = force[i,j]/mass # a = F/m
            
            if (finite_box) # If finite box, check are still inside and correct if not
                if (periodic) # For periodic, just change position to be in box
                    if (pos[i,j] < 0) # If no longer in box
                        pos[i,j] = box_size + (pos[i,j] % box_size)
                    elseif (pos[i,j] > box_size) # If no longer in box
                        pos[i,j] = pos[i,j] % box_size                        
                    end
                else # If not periodic, more complicated - reflects off walls
                    if (pos[i,j] < 0) # If no longer in box
                        pos[i,j] = -1*(pos[i,j])
                    elseif (pos[i,j] > box_size) # If no longer in box
                        pos[i,j] = box_size - pos[i,j]
                    end
                    vel[i,j] = -1*(vel[i,j])
                    acc[i,j] = -1*(acc[i,j])
                end
            end
        end
    end    
    # return pos, vel, acc # No need to return b/c is passed by reference
    return nothing # Modifica, C++ style
end

step_update! (generic function with 1 method)

In [21]:
#= The main program function, take in some parameters, returns a 3D matrix of positions for each time interval.

-> Input:
1. number_of_steps = Numero di iterazioni della simulazione, intero positivo [1, +∞)
2. dim = Numero di dimensioni dello spazio (2D o 3D), intero positivo, tipicamente [2,3]
3. box_size = Lunghezza di un lato del box di simulazione (quadrato o cubo), float positivo [1.0, +∞)
4. finite_box = Vero se il campo è finito <APPROFONDIRE, booleano [true, false]
5. periodic = Vero se la simulazione è periodica (pac-man), booleano [true, false]
6. part_num = Numero di particelle da simulare, intero positivo [1, +∞) 
7. dt = Grandezza del passo temporale, float positivo preferibilmente piccolo (0, +∞)
8. num_part_types = Numero di tipologie di particelle Intero positivo [1, +∞)
9. save_interval = Ogni quanti steps viene salvata la posizione, intero positivo [1, number_of_steps]
# Nuove variabili (?)
10. load_from_disk = Vero se si vuole caricare da disco i dati di una simulazione precedente (sovrascrive alcuni parametri)
11. save_on_disk = Vero se si vuole salvare su disco lo stato finale della simulazione

<- Output:
1. saved_positions = Sequenza temporale delle posizioni di ciascuna particella, matrice 3D, num_pos x part_num x dim
(num_pos = Numero di posizioni da salvare, funzione di number_of_steps e save_interval )
============================#
function dynamics_simulation(number_of_steps, dim, box_size, finite_box, periodic, part_num, dt, num_part_types, save_interval)
    # Inizializza strutture dati    
    # Costanti dipendenti dai parametri
    interaction_params = gen_interaction(num_part_types) # parametri di interazione, matrice quadrata num_part_types^2
    mass_parts = rand(Float32, num_part_types); # Masses of types of particles, massa per ogni tipo di particella, array
    part_types = rand(1:num_part_types, part_num) # Randomly assign type of each particle
      
    # Variabili aggiornate ad ogni iterazione
    pos = box_size*rand(Float32, part_num, dim) # Initialized to be randomly placed within a box (Usare funzione?)
    vel = Array{Float32}(undef, part_num, dim) #zeros(Float64, part_num, dim) # Initialized to zero
    acc = Array{Float32}(undef, part_num, dim) #zeros(Float64, part_num, dim) # Initialized to zero
    force = Array{Float32}(undef, part_num, dim) #zeros(Float64, part_num, dim) # Initialized to zero
    
    # Variabili per il salvataggio
    num_pos = floor(Int, number_of_steps/save_interval) + 1
    saved_positions = Array{Float32}(undef, num_pos, part_num, dim) #zeros(num_pos, part_num, dim)
    saved_velocities = Array{Float32}(undef, num_pos, part_num, dim) #zeros(num_pos, part_num, dim) # Save velocities with specified frequency
    saved_accelerations = Array{Float32}(undef, num_pos, part_num, dim) #zeros(num_pos, part_num, dim) # Save accelerations with specified frequency
    saved_forces = Array{Float32}(undef, num_pos, part_num, dim) #zeros(num_pos, part_num, dim) # Save forces with specified frequency
    
    # Calcolo delle forze agenti su ciascuna particella
    # Modifica: calcolo prima le forze e poi aggiorno con verlet direttamente nel loop
    # Loop per ciascuno step
    for t in 1:number_of_steps
        # Calcola forze agenti su ciascuna particella
        find_force!(force, part_num, dim, pos, vel, acc, part_types, interaction_params, mass_parts, periodic, box_size)
        # aggiornamento step
        step_update!(part_num, dim, pos, vel, acc, force, part_types, mass_parts, dt, box_size, finite_box, periodic)        
        # Salvataggi
        # Salva posizioni (TODO: funzione per il salvataggio save!(m2, m3) m2 matrice 2D, m3 matrice 3D
        # Se salvi su disco salva anche vel, acc e force
    end
    # Restituisci le posizioni per intervallo di tempo
    return saved_positions
end

dynamics_simulation (generic function with 1 method)

In [22]:
dynamics_simulation(100, 3, 10.0, true, true, 20, 0.01, 3, 2);
@time dynamics_simulation(100, 3, 10.0, true, true, 20, 0.01, 3, 2);

  0.000552 seconds (16 allocations: 50.984 KiB)


In [23]:
using BenchmarkTools
@benchmark dynamics_simulation(100, 3, 10.0, true, true, 20, 0.01, 3, 2)

BenchmarkTools.Trial: 
  memory estimate:  50.83 KiB
  allocs estimate:  12
  --------------
  minimum time:     486.400 μs (0.00% GC)
  median time:      838.799 μs (0.00% GC)
  mean time:        837.548 μs (0.21% GC)
  maximum time:     3.490 ms (73.89% GC)
  --------------
  samples:          5957
  evals/sample:     1